# Notebook principal para o pacote `robot_dynamics`

Este notebook age como um `main` executável no Colab (ou localmente), clonando o repositório antes de importar o pacote.
- Ajuste `REPO_URL` para o repositório Git que deseja usar.
- Após o clone, o caminho do repositório é adicionado ao `sys.path`, permitindo edições diretas nos módulos.
- Inclui um exemplo mínimo de cinemática e dinâmica para um robô planar de 2 DOF.


In [ ]:
import os
import subprocess
import sys

# Substitua pelo seu repositório (por exemplo, GitHub/Drive).
REPO_URL = "https://github.com/<usuario>/PFPython.git"
TARGET_DIR = "PFPython"

if not os.path.exists(TARGET_DIR):
    subprocess.run(["git", "clone", REPO_URL, TARGET_DIR], check=True)

repo_path = os.path.abspath(TARGET_DIR)
if repo_path not in sys.path:
    sys.path.insert(0, repo_path)

print(f"Usando repositório em: {repo_path}")


## Importar utilitários do pacote
Caso você edite o código em `robot_dynamics/`, basta reexecutar as células abaixo para refletir as mudanças.


In [ ]:
import sympy as sp

from robot_dynamics import (
    RobotModel,
    build_links_from_data,
    dynamics,
    forward_kinematics,
    parse_axis_order,
    spatial_jacobians,
)


## Exemplo rápido (2 DOF planar)
Os símbolos e parâmetros podem ser ajustados conforme necessário.


In [ ]:
# Símbolos
q1, q2 = sp.symbols("q1 q2")
dq1, dq2 = sp.symbols("dq1 dq2")
ddq1, ddq2 = sp.symbols("ddq1 ddq2")
a1, a2 = sp.symbols("a1 a2")
l1, l2 = sp.symbols("l1 l2")
m1, m2 = sp.symbols("m1 m2")
g = sp.symbols("g")

# Dados DH (theta, d, a, alpha) e eixos
dh_params = [
    (q1, 0, a1, 0),
    (q2, 0, a2, 0),
]
axes = parse_axis_order(["z", "z"])
masses = [m1, m2]
centers_of_mass = [
    (l1, 0, 0),
    (l2, 0, 0),
]

links = build_links_from_data(dh_params, axes, masses, centers_of_mass)
robot = RobotModel(links=links, grav=sp.Matrix([0, 0, -g]))

Ts, origins = forward_kinematics(robot)
Jvs, Jws = spatial_jacobians(robot, Ts, origins)

replacements, M, C, G, tau = dynamics(
    robot,
    [q1, q2],
    [dq1, dq2],
    [ddq1, ddq2],
    parallel=False,
)

T_ee = Ts[-1]
T_ee, Jvs[-1], Jws[-1], M, C, G, tau
